In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
import time
import re
from IPython.display import display, HTML

# Função genérica para configurar o navegador
def setup_browser(url):
    navegador = webdriver.Chrome()
    navegador.get(url)
    wait = WebDriverWait(navegador, 15)
    return navegador, wait

# Função para exibir os resultados de forma formatada em HTML
def display_results_formatted(results):
    html_content = "<h2>Resultados de Vagas de Emprego</h2><ul style='list-style-type:none;'>"
    
    for result in results:
        company, role_vagas = result.split(": ")
        role, vagas = role_vagas.split(" tem ")
        vagas = int(vagas.split(" ")[0])  # Extraindo o número de vagas

        # Definindo uma cor dependendo da quantidade de vagas
        color = "green" if vagas > 3 else "orange" if vagas > 0 else "red"

        # Adicionando o item à lista HTML
        html_content += f"<li><b style='color:{color};'>{company.strip()} - {role.strip()}:</b> <span style='color:{color};'>{vagas} vagas</span></li>"

    html_content += "</ul>"

    # Exibindo o conteúdo no Jupyter Notebook
    display(HTML(html_content))

# Função para buscar e contar vagas no Ifood
def fetch_ifood():
    navegador, wait = setup_browser("https://carreiras.ifood.com.br/")
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/section/section/section/div[1]/div[3]/div[1]/input')))
    input_element.send_keys('analista')
    input_element.send_keys(Keys.RETURN)

    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    expressions = ["analista de dados", "analista de bi", "data analyst", "analytics engineer"]
    results = {f"No IFOOD: '{expression}' tem {page_text.count(expression)} vagas.": page_text.count(expression) for expression in expressions}

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no Nubank
def fetch_nubank():
    navegador, wait = setup_browser("https://international.nubank.com.br/pt-br/carreiras/")
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="update_font_size"]/body/main/section[1]/div/div/div[1]/div/input')))
    input_element.send_keys('Analytics')
    input_element.send_keys(Keys.RETURN)

    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="update_font_size"]/body/main/section[1]/div/div/div[2]/div[2]')))
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    expressions = ["analytics engineer", "data analytics"]
    results = {f"No Nubank: '{expression}' tem {page_text.count(expression)} vagas.": page_text.count(expression) for expression in expressions}

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no C6 Bank
def fetch_c6bank():
    navegador, wait = setup_browser("https://job-boards.greenhouse.io/c6bank")
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="keyword-filter"]')))

    termos_pesquisa = ["Analista de Performance BI", "Engenharia de Dados", "Engenheira de Dados"]
    results = {}

    for termo in termos_pesquisa:
        input_element.clear()
        input_element.send_keys(termo)
        input_element.send_keys(Keys.RETURN)
        time.sleep(5)

        page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()
        results[f"No C6 Bank: '{termo}' tem {page_text.count(termo.lower())} vagas."] = page_text.count(termo.lower())

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no PicPay
def fetch_picpay():
    navegador, wait = setup_browser("https://picpay.com/oportunidades-de-emprego-e-carreiras/central-de-vagas")
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))).click()

    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/section[1]/div[3]/input')))
    input_element.send_keys('Analista de dados')

    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()
    results = {f"No PicPay: 'analista de dados' tem {page_text.count('analista de dados')} vagas.": page_text.count("analista de dados")}

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no Itaú
def fetch_itau():
    navegador, wait = setup_browser("https://carreiras.itau.com.br/tecnologia")
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page"]/header/div/div[1]/div[1]/div[2]/nav/button[1]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='advanced-search-keyword']"))).send_keys('Dados')
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='small-search-button']"))).click()

    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-results-list"]/ul')))
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    results = {f"No Itaú: 'dados' tem {page_text.count('dados')} vagas.": page_text.count("dados")}
    
    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no Santander
def fetch_santander():
    navegador, wait = setup_browser("https://santander.wd3.myworkdayjobs.com/pt-BR/SantanderCareers")

    # Clicar no campo de busca e inserir "Dados"
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='css-1i2ir9x']")))
    input_element.click()
    input_element.send_keys('Dados')

    # Clicar em "Local"
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mainContent"]/div/div[1]/fieldset/div[1]/button')))
    input_element.click()

    # Selecionar "Brasil"
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[1]/div/fieldset[1]/section/span/fieldset/div[1]/div/div/div[3]/div/div/label')))
    input_element.click()

    # Aplicar filtro de país
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/div/div/div/button')))
    input_element.click()

    # Clicar em pesquisar
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='css-1s27mqb']")))
    input_element.click()

    # Aguardar o carregamento das vagas
    time.sleep(6)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="mainContent"]/div/div[2]/section')))

    # Extrair o texto visível da página
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    # Normalizar texto removendo pontos e espaços extras
    normalized_text = re.sub(r'[\.\s]', '', page_text.lower())

    # Lista de expressões para contar
    expression_groups = {
        "Eng. de Dados": ["eng. de dados", "eng de dados"],
        "Analista de Dados": ["analista de dados"]
    }

    # Contar ocorrências combinadas
    results = {}
    for key, variations in expression_groups.items():
        count = sum(normalized_text.count(re.sub(r'[\.\s]', '', variation.lower())) for variation in variations)
        results[f"No Santander: '{key}' tem {count} vagas."] = count

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results


# Função principal para rodar as buscas simultaneamente
def main():
    fetch_functions = [fetch_ifood, fetch_nubank, fetch_c6bank, fetch_picpay, fetch_itau, fetch_santander]

    results = []

    with ThreadPoolExecutor(max_workers=len(fetch_functions)) as executor:
        future_to_function = {executor.submit(func): func.__name__ for func in fetch_functions}

        for future in future_to_function:
            try:
                result = future.result()
                results.extend(result.keys())  # Coletando as chaves dos resultados
            except Exception as e:
                print(f"Erro na execução de {future_to_function[future]}: {e}")

    # Exibindo os resultados no formato
    display_results_formatted(results)

if __name__ == "__main__":
    main()

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor
import time
import re
from IPython.display import display, HTML

# Função genérica para configurar o navegador
def setup_browser(url):
    navegador = webdriver.Chrome()
    navegador.get(url)
    wait = WebDriverWait(navegador, 15)
    return navegador, wait

# Função para exibir os resultados de forma formatada em HTML
def display_results_formatted(results):
    html_content = "<h2>Resultados de Vagas de Emprego</h2><ul style='list-style-type:none;'>"
    
    for result in results:
        company, role_vagas = result.split(": ")
        role, vagas = role_vagas.split(" tem ")
        vagas = int(vagas.split(" ")[0])  # Extraindo o número de vagas

        # Definindo uma cor dependendo da quantidade de vagas
        color = "green" if vagas > 3 else "orange" if vagas > 0 else "red"

        # Adicionando o item à lista HTML
        html_content += f"<li><b style='color:{color};'>{company.strip()} - {role.strip()}:</b> <span style='color:{color};'>{vagas} vagas</span></li>"

    html_content += "</ul>"

    # Exibindo o conteúdo no Jupyter Notebook
    display(HTML(html_content))

# Função para buscar e contar vagas no Ifood
def fetch_ifood():
    navegador, wait = setup_browser("https://carreiras.ifood.com.br/")
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/section/section/section/div[1]/div[3]/div[1]/input')))
    input_element.send_keys('analista')
    input_element.send_keys(Keys.RETURN)

    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    expressions = ["analista de dados", "analista de bi", "data analyst", "analytics engineer"]
    results = {f"Na Empresa 01: '{expression}' tem {page_text.count(expression)} vagas.": page_text.count(expression) for expression in expressions}

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no Nubank
def fetch_nubank():
    navegador, wait = setup_browser("https://international.nubank.com.br/pt-br/carreiras/")
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="update_font_size"]/body/main/section[1]/div/div/div[1]/div/input')))
    input_element.send_keys('Analytics')
    input_element.send_keys(Keys.RETURN)

    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="update_font_size"]/body/main/section[1]/div/div/div[2]/div[2]')))
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    expressions = ["analytics engineer", "data analytics"]
    results = {f"Na Empresa 02: '{expression}' tem {page_text.count(expression)} vagas.": page_text.count(expression) for expression in expressions}

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no C6 Bank
def fetch_c6bank():
    navegador, wait = setup_browser("https://job-boards.greenhouse.io/c6bank")
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="keyword-filter"]')))

    termos_pesquisa = ["Analista de Performance BI", "Engenharia de Dados", "Engenheira de Dados"]
    results = {}

    for termo in termos_pesquisa:
        input_element.clear()
        input_element.send_keys(termo)
        input_element.send_keys(Keys.RETURN)
        time.sleep(5)

        page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()
        results[f"Na Empresa 03: '{termo}' tem {page_text.count(termo.lower())} vagas."] = page_text.count(termo.lower())

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no PicPay
def fetch_picpay():
    navegador, wait = setup_browser("https://picpay.com/oportunidades-de-emprego-e-carreiras/central-de-vagas")
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))).click()

    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/main/section[1]/div[3]/input')))
    input_element.send_keys('Analista de dados')

    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()
    results = {f"Na Empresa 04: 'analista de dados' tem {page_text.count('analista de dados')} vagas.": page_text.count("analista de dados")}

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no Itaú
def fetch_itau():
    navegador, wait = setup_browser("https://carreiras.itau.com.br/tecnologia")
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page"]/header/div/div[1]/div[1]/div[2]/nav/button[1]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='advanced-search-keyword']"))).send_keys('Dados')
    wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='small-search-button']"))).click()

    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-results-list"]/ul')))
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    results = {f"Na Empresa 05: 'dados' tem {page_text.count('dados')} vagas.": page_text.count("dados")}
    
    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results

# Função para buscar e contar vagas no Santander
def fetch_santander():
    navegador, wait = setup_browser("https://santander.wd3.myworkdayjobs.com/pt-BR/SantanderCareers")

    # Clicar no campo de busca e inserir "Dados"
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='css-1i2ir9x']")))
    input_element.click()
    input_element.send_keys('Dados')

    # Clicar em "Local"
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="mainContent"]/div/div[1]/fieldset/div[1]/button')))
    input_element.click()

    # Selecionar "Brasil"
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[1]/div/fieldset[1]/section/span/fieldset/div[1]/div/div/div[3]/div/div/label')))
    input_element.click()

    # Aplicar filtro de país
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/div/div/div/button')))
    input_element.click()

    # Clicar em pesquisar
    input_element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@class='css-1s27mqb']")))
    input_element.click()

    # Aguardar o carregamento das vagas
    time.sleep(6)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="mainContent"]/div/div[2]/section')))

    # Extrair o texto visível da página
    page_text = navegador.find_element(By.TAG_NAME, "body").text.lower()

    # Normalizar texto removendo pontos e espaços extras
    normalized_text = re.sub(r'[\.\s]', '', page_text.lower())

    # Lista de expressões para contar
    expression_groups = {
        "Eng. de Dados": ["eng. de dados", "eng de dados"],
        "Analista de Dados": ["analista de dados"]
    }

    # Contar ocorrências combinadas
    results = {}
    for key, variations in expression_groups.items():
        count = sum(normalized_text.count(re.sub(r'[\.\s]', '', variation.lower())) for variation in variations)
        results[f"Na Empresa 06: '{key}' tem {count} vagas."] = count

    # Filtrando resultados com contagem 0
    results = {key: value for key, value in results.items() if value > 0}

    navegador.quit()
    return results


# Função principal para rodar as buscas simultaneamente
def main():
    fetch_functions = [fetch_ifood, fetch_nubank, fetch_c6bank, fetch_picpay, fetch_itau, fetch_santander]

    results = []

    with ThreadPoolExecutor(max_workers=len(fetch_functions)) as executor:
        future_to_function = {executor.submit(func): func.__name__ for func in fetch_functions}

        for future in future_to_function:
            try:
                result = future.result()
                results.extend(result.keys())  # Coletando as chaves dos resultados
            except Exception as e:
                print(f"Erro na execução de {future_to_function[future]}: {e}")

    # Exibindo os resultados no formato
    display_results_formatted(results)

if __name__ == "__main__":
    main()